In [5]:
import pandas as pd
import numpy as np
import wikipedia as wp
import requests
import os
import urllib.request
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


This is a WorkBook dedicated for MyCapStone Project

In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")

print(soup.title.string)
right_table=soup.find('table', class_='wikitable sortable')
AA = []
BB = []
CC = []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0:
        AA.append(cells[0].find(text=True).rstrip("\n"))
        BB.append(cells[1].find(text=True).rstrip("\n"))
        CC.append(cells[2].find(text=True).rstrip("\n"))
        
Wiki_DF_1 = pd.DataFrame(AA,columns=['Postalcode'])
Wiki_DF_1['Borough']=BB
Wiki_DF_1['Neighbourhood']=CC
print("Raw Table Size from URL : " + str(Wiki_DF_1.shape))
Wiki_DF_1.to_csv(os.getcwd() + "\\1_MyTorontoPC_RAW.csv")

A = []
B = []
C = []
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0:
        if cells[1].find(text=True).rstrip("\n") != "Not assigned":
            A.append(cells[0].find(text=True).rstrip("\n"))
            B.append(cells[1].find(text=True).rstrip("\n"))
            if cells[2].find(text=True).rstrip("\n") == "Not assigned":
                C.append(cells[1].find(text=True).rstrip("\n"))
            else:
                C.append(cells[2].find(text=True).rstrip("\n"))
        
Wiki_DF_2 = pd.DataFrame(A,columns=['Postalcode'])
Wiki_DF_2['Borough']=B
Wiki_DF_2['Neighbourhood']=C
Wiki_DF_2.to_csv(os.getcwd() + "\\2_MyTorontoPC_NA.csv")
print("Table Size after removing row's containing Borough as \"Not assigned\" : " + str(Wiki_DF_2.shape))

Wiki_DF_3 = Wiki_DF_2.groupby(['Postalcode','Borough'], as_index = False).agg({'Neighbourhood': ','.join})
Wiki_DF_3.to_csv(os.getcwd() + "\\3_MyTorontoData_GROUPED.csv")
print("Table Size after grouping row by same postalcode : " + str(Wiki_DF_3.shape))

List of postal codes of Canada: M - Wikipedia
Raw Table Size from URL : (287, 3)
Table Size after removing row's containing Borough as "Not assigned" : (210, 3)
Table Size after grouping row by same postalcode : (103, 3)


<h1>MarkDown</h1><br>
All Requested Actions Were Completed <br>
Queried Table from a Wikipedia Link into a dataframe <br>
Processed it to ensure if Borough is Not Assigned then its data is not taken into account

In [36]:
import geocoder # import geocoder
import io
url="http://cocl.us/Geospatial_data"
Geospatial_data=requests.get(url).content

A = []
B = []
C = []
D = []
E = []

Wiki_DF = Wiki_DF_3
Wiki_PostLatLon = pd.read_csv(io.StringIO(Geospatial_data.decode('utf-8')))
for index, row in Wiki_DF.iterrows():
    A.append(row['Postalcode'])
    B.append(row['Borough'])
    C.append(row['Neighbourhood'])
    
    #location = None
    #while(location is None):
    #    geolocator = Nominatim(user_agent='MyCapStone')
    #    location = geolocator.geocode({"postalcode": row['Borough']})
    LatLon = Wiki_PostLatLon[Wiki_PostLatLon["Postal Code"] == row['Postalcode']]
    D.append(LatLon["Latitude"].values[0])
    E.append(LatLon["Longitude"].values[0])

Wiki_DF_4 = pd.DataFrame(A,columns=['Postalcode'])
Wiki_DF_4['Borough']=B
Wiki_DF_4['Neighbourhood']=C
Wiki_DF_4['Latitude']=D
Wiki_DF_4['Longitude']=E

Wiki_DF_4.to_csv(os.getcwd() + "\\3_MyTorontoData_LATLON.csv")
print("Done!")

Done!


In [51]:
CLIENT_ID = 'ARRNPU155D51DSX4JHDRNJLSAR3GS2JB0BZJX1N550ACFA5A' # your Foursquare ID
CLIENT_SECRET = 'OGUGTHQP15HJCC31I3QXXEK2FCAESKJV5HMCINXK023514EQ' # your Foursquare Secret
VERSION = '20191231' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Index = 1
Wiki_DF_4.loc[Index, 'Neighbourhood']
neighborhoods_LAT = Wiki_DF_4.loc[Index, 'Latitude'] # neighborhood latitude value
neighborhoods_LON = Wiki_DF_4.loc[Index, 'Longitude'] # neighborhood longitude value
neighborhoods_NAME = Wiki_DF_4.loc[Index, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhoods_NAME, 
                                                               neighborhoods_LAT, 
                                                               neighborhoods_LON))
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhoods_LAT, 
    neighborhoods_LON, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: ARRNPU155D51DSX4JHDRNJLSAR3GS2JB0BZJX1N550ACFA5A
CLIENT_SECRET:OGUGTHQP15HJCC31I3QXXEK2FCAESKJV5HMCINXK023514EQ
Latitude and longitude values of Highland Creek,Rouge Hill,Port Union are 43.7845351, -79.16049709999999.


{'meta': {'code': 200, 'requestId': '5e6881171a4b0a001b7a61e2'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.7890351045, 'lng': -79.15427558741125},
   'sw': {'lat': 43.7800350955, 'lng': -79.16671861258872}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5b19cb3bacc5f5002c693e5c',
       'name': 'Great Shine Window Cleaning',
       'location': {'address': '11 Clyde Rd',
        'lat': 43.7831452,
        'lng': -79.1574307,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7831452

In [52]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
import json
from pandas import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

                          name    categories        lat        lng
0  Great Shine Window Cleaning  Home Service  43.783145 -79.157431
1        Royal Canadian Legion           Bar  43.782533 -79.163085
2 venues were returned by Foursquare.


In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
Toronto_data = Wiki_DF_4[Wiki_DF_4['Borough'].str.contains("Toronto")].reset_index(drop=True)
#print(Toronto_data)
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )
print(Toronto_venues.shape)
Toronto_venues.head()
Toronto_data.groupby('Neighbourhood').count()


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

,Postalcode,Borough,Latitude,Longitude
Neighbourhood,,,,
"Adelaide,King,Richmond",1,1,1,1
Berczy Park,1,1,1,1
"Brockton,Exhibition Place,Parkdale Village",1,1,1,1
Business Reply Mail Processing Centre 969 Eastern,1,1,1,1
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",1,1,1,1
"Cabbagetown,St. James Town",1,1,1,1
Central Bay Street,1,1,1,1
"Chinatown,Grange Park,Kensington Market",1,1,1,1
Christie,1,1,1,1
